# **Session 7**

In [ ]:
# Check GPU availability
!nvidia-smi

Sat May 31 11:58:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Checkpoint Directory
import os
DRIVE_CHECKPOINT_DIR = '/content/drive/MyDrive/YOLOv12_ASD_Checkpoints'
if not os.path.exists(DRIVE_CHECKPOINT_DIR):
    raise FileNotFoundError("Make sure the shortcut 'YOLOv12_ASD_Checkpoints' is in your MyDrive.")
os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)

In [ ]:
# Set environment flag
os.environ["NO_ALBUMENTATIONS_UPDATE"] = "1"

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn albumentations comet_ml

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 113.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Imports after dependencies
import random
import shutil
import numpy as np
import torch
import cv2
import albumentations as A
from tqdm import tqdm
from ultralytics import YOLO
from google.colab import userdata
import comet_ml
from roboflow import Roboflow
import time
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/yolov12/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


In [ ]:
# Verify shared folder access
print(f"Using shared folder: {DRIVE_CHECKPOINT_DIR}")
!ls {DRIVE_CHECKPOINT_DIR}

Using shared folder: /content/drive/MyDrive/YOLOv12_ASD_Checkpoints
best.pt			    experiment_id_session4.txt	experiment_id.txt
experiment_id_session2.txt  experiment_id_session5.txt	last.pt
experiment_id_session3.txt  experiment_id_session6.txt


In [ ]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
# Set environment variables
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")
os.environ["COMET_API_KEY"] = userdata.get("COMET_API_KEY")

In [ ]:
# Load dataset from Roboflow
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])
project = rf.workspace("noor-fatima-vfkuh").project("asd-eobeu")
dataset = project.version(18).download("yolov8")
dataset_location = dataset.location

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ASD-18 in yolov8:: 100%|██████████| 26274/26274 [00:06<00:00, 3827.51it/s]


In [ ]:
# Verify dataset structure
print("Verifying dataset structure:")
!ls {dataset_location}
for folder in ['train', 'valid', 'test']:
    if os.path.exists(os.path.join(dataset_location, folder, 'images')):
        print(f"{folder}/images found")
    else:
        print(f"Warning: {folder}/images not found")

Verifying dataset structure:
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid
train/images found
valid/images found
test/images found


In [ ]:
# Update data.yaml
data_yaml_path = f"{dataset_location}/data.yaml"
try:
    with open(data_yaml_path, 'r') as file:
        lines = file.readlines()
    lines = [line for line in lines if not line.startswith(('test:', 'train:', 'val:'))]
    lines.extend([
        "test: ./test/images\n",
        "train: ./train/images\n",
        "val: ./valid/images\n"
    ])
    with open(data_yaml_path, 'w') as file:
        file.writelines(lines)
    print("data.yaml updated successfully")
    with open(data_yaml_path, 'r') as file:
        print("data.yaml contents:")
        print(file.read())
except Exception as e:
    print(f"Error updating data.yaml: {e}")
    raise

data.yaml updated successfully
data.yaml contents:
names:
- Lack_social_skill
- Stimming
- aligning_objects
- avoid_eye_contact
- awkward_posture
- clapping
- closing_eyes
- continuous_moving
- exaggerated_expression
- face_rubbing
- finger_biting
- finger_flapping
- finger_smelling
- finger_tapping
- hand_flapping
- hand_leading
- head_banging
- holding_objects
- jumping
- lack_of_awareness
- lack_of_response
- moving_hand_upforth
- normal
- repetitive_action
- rocking
- rubbing_eyes
- rubbing_hands
- rubbing_nose
- rubbing_objects
- smelling
- spinning
- tapping_head
- weird_expression
- weird_laugh
nc: 34
roboflow:
  license: CC BY 4.0
  project: asd-eobeu
  url: https://universe.roboflow.com/noor-fatima-vfkuh/asd-eobeu/dataset/18
  version: 18
  workspace: noor-fatima-vfkuh
test: ./test/images
train: ./train/images
val: ./valid/images



In [ ]:
# Apply augmentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.4, brightness_limit=0.2, contrast_limit=0.2),
    A.Rotate(limit=20, p=0.5),
    A.Resize(640, 640),
])

def apply_augmentation_to_folder(img_folder):
    for file in tqdm(os.listdir(img_folder), desc="Augmenting images"):
        path = os.path.join(img_folder, file)
        try:
            img = cv2.imread(path)
            if img is None:
                continue
            augmented = transform(image=img)['image']
            cv2.imwrite(path, augmented)
        except Exception as e:
            print(f"Error processing {file}: {e}")

apply_augmentation_to_folder(f"{dataset_location}/train/images")

Augmenting images: 100%|██████████| 11672/11672 [01:20<00:00, 144.60it/s]


In [ ]:
# Define function to log metrics to Comet ML
def log_metrics_to_comet(experiment, session_number, results_csv_path):
    try:
        if not os.path.exists(results_csv_path):
            experiment.log_text(f"Results CSV not found at {results_csv_path} for session {session_number}")
            return
        results_df = pd.read_csv(results_csv_path)
        results_df.columns = results_df.columns.str.strip()
        global_epoch_offset = 100  # Session 1: 10, Session 2: 15, Session 3: 15, Session 4: 20, Session 5: 20, Session 6: 20
        for i, row in results_df.iterrows():
            metrics = {
                "train/box_loss": float(row["train/box_loss"]) if not pd.isna(row["train/box_loss"]) else 0.0,
                "train/cls_loss": float(row["train/cls_loss"]) if not pd.isna(row["train/cls_loss"]) else 0.0,
                "train/dfl_loss": float(row["train/dfl_loss"]) if not pd.isna(row["train/dfl_loss"]) else 0.0,
                "val/box_loss": float(row["val/box_loss"]) if not pd.isna(row["val/box_loss"]) else 0.0,
                "val/cls_loss": float(row["val/cls_loss"]) if not pd.isna(row["val/cls_loss"]) else 0.0,
                "val/dfl_loss": float(row["val/dfl_loss"]) if not pd.isna(row["val/dfl_loss"]) else 0.0,
                "metrics/precision(B)": float(row["metrics/precision(B)"]),
                "metrics/recall(B)": float(row["metrics/recall(B)"]),
                "metrics/mAP50(B)": float(row["metrics/mAP50(B)"]),
                "metrics/mAP50-95(B)": float(row["metrics/mAP50-95(B)"]),
                "lr/pg0": float(row["lr/pg0"]),
                "lr/pg1": float(row["lr/pg1"]),
                "lr/pg2": float(row["lr/pg2"])
            }
            experiment.log_metrics(metrics, step=global_epoch_offset + i + 1)
        print(f"Metrics logged to Comet ML for session {session_number}")
    except Exception as e:
        print(f"Error logging metrics from results.csv: {e}")
        experiment.log_text(f"Error logging metrics for session {session_number}: {str(e)}")

In [ ]:
# Initialize Comet ML
try:
    comet_ml.login(api_key=userdata.get("COMET_API_KEY"))
except Exception as e:
    print(f"Error logging into Comet ML: {e}")
    raise

session_number = 7  # Session 7

COMET INFO: Valid Comet API Key saved in /content/drive/MyDrive/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
# Securely validate API keys
try:
    if not userdata.get("COMET_API_KEY"):
        raise ValueError("COMET_API_KEY not set in Colab Secrets.")
except Exception as e:
    print(f"Error validating COMET_API_KEY: {e}")
    raise

In [ ]:
# Create new experiment for Session 7
try:
    experiment = comet_ml.Experiment(
        project_name="yolov12-autism-detection",
        workspace="salamafazlul"
    )
    with open(os.path.join(DRIVE_CHECKPOINT_DIR, 'experiment_id_session7.txt'), 'w') as f:
        f.write(experiment.id)
    experiment.log_text(f"Experiment ID: {experiment.id}")
except Exception as e:
    print(f"Error creating Comet ML experiment: {e}")
    raise

try:
    experiment.set_name("asd_yolov12_session_7")
    experiment.log_parameters({
        "epochs": 20,
        "total_epochs": 120,
        "imgsz": 640,
        "batch": 8,
        "device": 0,
        "model": "yolov12m.yaml",
        "optimizer": "AdamW",
        "lr0": 0.00015,
        "mosaic": 1.0,
        "copy_paste": 0.1,
        "workers": 2,
        "session_number": session_number
    })
except Exception as e:
    print(f"Error setting Comet ML parameters: {e}")
    raise

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/salamafazlul/yolov12-autism-detection/5fe1a0192f064eb399013f27cfe5c3df

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [ ]:
# Initialize model from Session 6 checkpoint
try:
    checkpoint_path = os.path.join(DRIVE_CHECKPOINT_DIR, "last.pt")
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint 'last.pt' from Session 6 not found in {DRIVE_CHECKPOINT_DIR}")
    print(f"Resuming training from checkpoint: {checkpoint_path}")
    model = YOLO(checkpoint_path)
except Exception as e:
    print(f"Error loading model checkpoint: {e}")
    experiment.log_text(f"Error loading model for session {session_number}: {str(e)}")
    raise

Resuming training from checkpoint: /content/drive/MyDrive/YOLOv12_ASD_Checkpoints/last.pt


In [ ]:
# Train model
start_time = time.time()
try:
    model.train(
        data=data_yaml_path,
        epochs=20,
        imgsz=640,
        batch=8,
        device=0,
        amp=True,
        cache=False,
        save_period=4,
        name=f"asd_yolov12_session_{session_number}",
        project="autism-detection",
        exist_ok=True,
        verbose=True,
        patience=10,
        optimizer="AdamW",
        lr0=0.00015,
        cos_lr=True,
        mosaic=1.0,
        copy_paste=0.1,
        workers=2,
        resume=False
    )
except KeyboardInterrupt:
    print("Training interrupted, logging metrics and saving checkpoint...")
    try:
        results_csv_path = f"autism-detection/asd_yolov12_session_{session_number}/results.csv"
        log_metrics_to_comet(experiment, session_number, results_csv_path)
        model.save(os.path.join(DRIVE_CHECKPOINT_DIR, "last.pt"))
        experiment.log_model("yolov12_asd_last", os.path.join(DRIVE_CHECKPOINT_DIR, "last.pt"))
        experiment.end()
        print("Checkpoint saved to Google Drive")
    except Exception as e:
        print(f"Error during interrupt handling: {e}")
        experiment.log_text(f"Error during interrupt for session {session_number}: {str(e)}")
    raise
except Exception as e:
    print(f"Training error: {e}")
    experiment.log_text(f"Training error for session {session_number}: {str(e)}")
    raise

print(f"Session {session_number} took {(time.time() - start_time) / 60:.2f} minutes")

New https://pypi.org/project/ultralytics/8.3.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv12_ASD_Checkpoints/last.pt, data=/content/ASD-18/data.yaml, epochs=20, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=4, cache=False, device=0, workers=2, project=autism-detection, name=asd_yolov12_session_7, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, class

100%|██████████| 755k/755k [00:00<00:00, 112MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     37120  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2, 1, 2]         
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    147968  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2, 1, 4]        
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2664960  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 4]        
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 5.26M/5.26M [00:00<00:00, 38.2MB/s]


AMP: checks passed ✅


...


20 epochs completed in 4.139 hours.
Optimizer stripped from autism-detection/asd_yolov12_session_7/weights/last.pt, 39.8MB
Optimizer stripped from autism-detection/asd_yolov12_session_7/weights/best.pt, 39.8MB

Validating autism-detection/asd_yolov12_session_7/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 402 layers, 19,602,742 parameters, 0 gradients, 59.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:15<00:00,  2.88it/s]


                   all        730        743      0.593      0.609      0.605      0.396
     Lack_social_skill         18         18      0.536      0.771      0.715      0.439
              Stimming         17         17      0.249      0.941      0.266      0.199
      aligning_objects         16         16      0.741       0.75      0.716      0.453
     avoid_eye_contact         21         21      0.354      0.418      0.398      0.239
       awkward_posture         21         21      0.416      0.619      0.452      0.235
              clapping          8          8      0.592        0.5      0.618      0.466
          closing_eyes         13         13       0.51      0.322      0.412      0.336
     continuous_moving         15         15      0.649      0.667      0.814      0.469
exaggerated_expression         17         17      0.491      0.588      0.566      0.403
          face_rubbing         16         16      0.855      0.875       0.93      0.461
         finger_bitin

COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : asd_yolov12_session_7
COMET INFO:     url                   : https://www.comet.com/salamafazlul/yolov12-autism-detection/5fe1a0192f064eb399013f27cfe5c3df
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [41]               : (2.4141407108110133e-06, 0.00014633250654822678)
COMET INFO:     lr/pg1 [41]               : (2.4141407108110133e-06, 0.00014633250654822678)
COMET INFO:     lr/pg2 [41]               : (2.4141407108110133e-06, 0.00014633250654822678)
COMET INFO:     metrics/mAP50(B) [42]     : (0.51748, 0.60779)


Session 7 took 249.37 minutes


In [ ]:
# Log metrics after training
results_csv_path = f"autism-detection/asd_yolov12_session_{session_number}/results.csv"
log_metrics_to_comet(experiment, session_number, results_csv_path)

Metrics logged to Comet ML for session 7


In [ ]:
# Save final model for this session
try:
    model.save(os.path.join(DRIVE_CHECKPOINT_DIR, "last.pt"))
    experiment.log_model("yolov12_asd_last", os.path.join(DRIVE_CHECKPOINT_DIR, "last.pt"))
    if os.path.exists(os.path.join("autism-detection", f"asd_yolov12_session_{session_number}", "weights", "best.pt")):
        shutil.copy(
            os.path.join("autism-detection", f"asd_yolov12_session_{session_number}", "weights", "best.pt"),
            os.path.join(DRIVE_CHECKPOINT_DIR, "best.pt")
        )
        experiment.log_model("yolov12_asd_best", os.path.join(DRIVE_CHECKPOINT_DIR, "best.pt"))
except Exception as e:
    print(f"Error saving model: {e}")
    experiment.log_text(f"Error saving model for session {session_number}: {str(e)}")
    raise

In [ ]:
# Validate metrics
print("Running validation to verify metrics...")
model.val(data=data_yaml_path, imgsz=640, batch=8, cache=False)

Running validation to verify metrics...
Ultralytics 8.3.63 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 402 layers, 19,602,742 parameters, 0 gradients, 59.6 GFLOPs


val: Scanning /content/ASD-18/valid/labels.cache... 730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 730/730 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/ASD-18/valid/images/Screenshot-2024-08-26-111148_jpg.rf.02242ccd47e086b3277d53a649f9c502.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/ASD-18/valid/images/Screenshot-2024-08-26-111148_jpg.rf.6ff0fd368a89c33f50c454bb6868b5ff.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:39<00:00,  2.34it/s]


                   all        730        743      0.592      0.608      0.605      0.395
     Lack_social_skill         18         18      0.535      0.769      0.715      0.436
              Stimming         17         17      0.249      0.941      0.262      0.197
      aligning_objects         16         16      0.741       0.75      0.716      0.453
     avoid_eye_contact         21         21      0.323      0.381      0.396      0.237
       awkward_posture         21         21      0.417      0.619      0.452      0.231
              clapping          8          8      0.593        0.5      0.618      0.466
          closing_eyes         13         13       0.51      0.321      0.413      0.337
     continuous_moving         15         15      0.648      0.667      0.814      0.469
exaggerated_expression         17         17      0.491      0.588      0.566        0.4
          face_rubbing         16         16      0.856      0.875       0.93      0.462
         finger_bitin

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc448f57c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0

In [ ]:
# End Comet experiment
try:
    experiment.end()
except Exception as e:
    print(f"Error ending Comet ML experiment: {e}")
    experiment.log_text(f"Error ending experiment for session {session_number}: {str(e)}")